In [2]:
import pandas as pd
df=pd.read_csv('4dataSeH_model.csv')

In [3]:
from sklearn.linear_model import LinearRegression
from testset_prediction import testset_prediction as tsp
from sequential_selection import stepwise_selection as sq
from loo import loo
from rm2 import rm2
reg=LinearRegression()

def process(tr,trX,Xts,y,yts,seed):
    lt=[]
    ls=['r2','neg_mean_absolute_error','neg_mean_poisson_deviance','neg_mean_gamma_deviance']
    #ls=['r2']
    lf=[0,5]
    #lf=[0]
    l1,l2,l3,l4,l5,l6,l7=[],[],[],[],[],[],[]
    for i in ls:
        for j in lf:
            sqs=sq(trX,y,8,True,True,i,j)
            try:
               a1,b1=sqs.fit_()
               reg.fit(tr[a1],y)
               r2tr=reg.score(tr[a1],y)
               cv=loo(tr[a1],y,tr)
               print(a1)
               c,m,l=cv.cal()
               r2pr,r2pr2,RMSEP=testpred(Xts[a1],yts,reg,m)
               rm2tr,drm2tr=rm2(y,l).fit()
               ytspr=pd.DataFrame(reg.predict(Xts[a1]))
               rm2ts,drm2ts=rm2(yts,ytspr).fit()
            except ValueError:
                c=0
                r2pr=0
                rm2tr=0
                rm2ts=0         
            #print(c,m,l)
            #print(a1)
            l1.append(i) 
            l2.append(j)  
            l3.append(c)
            l4.append(r2pr)
            l6.append(rm2tr)
            l7.append(rm2ts)
            l5.append(seed)
    Dict=dict([('random_seed', l5),('score', l1),('fold', l2),('Q2LOO', l3), ('R2Pred', l4), ('rm2tr', l6), ('rm2ts', l7)])
    print(Dict)
    table=pd.DataFrame(Dict)
    return table
    #tbname='Results_table.csv'
    #table.to_csv('Results_table.csv',index=False)  

In [4]:
from sklearn.feature_selection import VarianceThreshold
def correlation(X,cthreshold):
    col_corr = set() # Set of all the names of deleted columns
    corr_matrix = X.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if (abs(corr_matrix.iloc[i, j]) > float(cthreshold)) and (corr_matrix.columns[j] not in col_corr):
                colname = corr_matrix.columns[i] # getting the name of column
                col_corr.add(colname)
                if colname in X.columns:
                   del X[colname] # deleting the column from the dataset
    return X   

def variance(X,threshold):
    sel = VarianceThreshold(threshold=(threshold* (1 - threshold)))
    sel_var=sel.fit_transform(X)
    X=X[X.columns[sel.get_support(indices=True)]]    
    return X


def pretreat(X,cthreshold,vthreshold):
    X=correlation(X,cthreshold)
    X=variance(X,vthreshold)
    return X

In [5]:
def testpred(Xts,yts,model,trav):
    ytspr=pd.DataFrame(model.predict(Xts))
    ytspr.columns=['Pred']
    tsdf=pd.concat([yts,pd.DataFrame(ytspr)],axis=1)
    tsdf.columns=['Active','Predict']
    tsdf['Aver']=trav
    tsdf['Aver2']=tsdf['Predict'].mean()
    tsdf['diff']=tsdf['Active']-tsdf['Predict']
    tsdf['diff2']=tsdf['Active']-tsdf['Aver']
    tsdf['diff3']=tsdf['Active']-tsdf['Aver2']
    r2pr=1-((tsdf['diff']**2).sum()/(tsdf['diff2']**2).sum())
    r2pr2=1-((tsdf['diff']**2).sum()/(tsdf['diff3']**2).sum())
    RMSEP=((tsdf['diff']**2).sum()/tsdf.shape[0])**0.5
    return r2pr,r2pr2,RMSEP

In [6]:
import pandas as pd
X=df.iloc[:,2:]
Xpt=pretreat(X,0.99,0.0001)

In [7]:
from sklearn.model_selection import train_test_split
import pandas as pd
#perc=secondEntryTabOne.get()
#df=pd.read_csv('merged2_pparg_model2.csv')
#X=df.iloc[:,1:]
#pt=pd.read_csv('pt_train_0.99_0.0001.csv')
Xpt2=Xpt.columns.tolist()[1:]
perc=0.2
nls=[]
ml=[3,20,42]
#ml=[20]
for rs in ml:
    #perc=int(secondEntryTabOne.get())
    #pc=int(100/perc)
    #sp=int(secondEntryTabOne_x.get())
    #filem=df.sort_values(df.iloc[:,1:2].columns[0],ascending=False)
    #ts=filem.iloc[(sp-1)::pc, :]
    #tr=filem.drop(ts.index.values)
    #tr.to_csv('tr.csv', index=False)
    a,b= train_test_split(df,test_size=perc, random_state=rs)
    tr=pd.DataFrame(a)
    ts=pd.DataFrame(b)
    tr=tr.reset_index().drop('index', axis=1)
    ts=ts.reset_index().drop('index', axis=1)
    trX=tr[Xpt2]
    Xts=ts[Xpt2]
    #print(trX.shape)
    y=tr.iloc[:,1:2]
    tr1=pd.concat([tr.iloc[:,0:2],trX],axis=1)
    ts1=pd.concat([ts.iloc[:,0:2],Xts],axis=1)
    #tr1.to_csv('tr.csv', index=False)
    #ts1.to_csv('ts.csv', index=False)
    yts=ts.iloc[:,1:2] 
    table=process(tr,trX,Xts,y,yts,rs)
    nls.append(table)
    fd=pd.concat(nls, axis=0)

C:\Users\amitc\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


['VE2_B(e)', 'ATS1m', 'MATS4s', 'SpMax8_Bh(v)', 'minssCH2', 'CATS2D_05_DL', 'SHED_AA', 'B06[C-S]']


C:\Users\amitc\Documents\subham_work\SFS-QSAR-tool-master\loo.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  dataf['Del_Res']=dataf[self.y.columns[0]]-dataf['Pred_loo']
C:\Users\amitc\Documents\subham_work\SFS-QSAR-tool-master\loo.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  dataf['aver']=dataf[self.y.columns[0]].mean()
C:\Users\amitc\Documents\subham_work\SFS-QSAR-tool-master\loo.py:35: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has

['IAC', 'SpMin5_Bh(p)', 'P_VSA_LogP_2', 'C-014', 'CATS2D_09_AA', 'SHED_AN', 'T(Cl..Cl)', 'F07[N-N]']


C:\Users\amitc\Documents\subham_work\SFS-QSAR-tool-master\loo.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  dataf['Del_Res']=dataf[self.y.columns[0]]-dataf['Pred_loo']
C:\Users\amitc\Documents\subham_work\SFS-QSAR-tool-master\loo.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  dataf['aver']=dataf[self.y.columns[0]].mean()
C:\Users\amitc\Documents\subham_work\SFS-QSAR-tool-master\loo.py:35: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has

...

C:\Users\amitc\Documents\subham_work\SFS-QSAR-tool-master\loo.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  dataf['Del_Res']=dataf[self.y.columns[0]]-dataf['Pred_loo']
C:\Users\amitc\Documents\subham_work\SFS-QSAR-tool-master\loo.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  dataf['aver']=dataf[self.y.columns[0]].mean()
C:\Users\amitc\Documents\subham_work\SFS-QSAR-tool-master\loo.py:35: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has

['Ho_D/Dt', 'ATS1m', 'Eig06_EA(bo)', 'CATS2D_05_DL', 'CATS2D_00_AA', 'B03[O-S]', 'F09[N-O]', 'MDEN-33']


C:\Users\amitc\Documents\subham_work\SFS-QSAR-tool-master\loo.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  dataf['Del_Res']=dataf[self.y.columns[0]]-dataf['Pred_loo']
C:\Users\amitc\Documents\subham_work\SFS-QSAR-tool-master\loo.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  dataf['aver']=dataf[self.y.columns[0]].mean()
C:\Users\amitc\Documents\subham_work\SFS-QSAR-tool-master\loo.py:35: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has

['ATS1m', 'MATS2i', 'GATS2i', 'SpMin7_Bh(e)', 'Eig06_EA(bo)', 'CATS2D_05_DL', 'B05[S-Cl]', 'MDEC-34']
{'random_seed': [42, 42, 42, 42, 42, 42, 42, 42], 'score': ['r2', 'r2', 'neg_mean_absolute_error', 'neg_mean_absolute_error', 'neg_mean_poisson_deviance', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'neg_mean_gamma_deviance'], 'fold': [0, 5, 0, 5, 0, 5, 0, 5], 'Q2LOO': [0.7576182321876062, 0.7751261172779991, 0.7378783689437368, 0.7402372305042726, 0.7564548619175453, 0.7629895046427984, 0.7479211041733324, 0.7359566726746893], 'R2Pred': [0.7527798210383531, 0.7847126574548727, 0.6780598097403294, 0.6967038141346803, 0.7365621852979252, 0.6833533246125878, 0.6721544204614247, 0.7627234396502427], 'rm2tr': [0.6678698002399808, 0.690069920488863, 0.6417493598756399, 0.6445291891025198, 0.6668692858891385, 0.6745001268196883, 0.6548268727209827, 0.6397012639940701], 'rm2ts': [0.6457501948615849, 0.7022997478274842, 0.5700286870867378, 0.5729169870059672, 0.6230247285323539, 0.

C:\Users\amitc\Documents\subham_work\SFS-QSAR-tool-master\loo.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  dataf['Del_Res']=dataf[self.y.columns[0]]-dataf['Pred_loo']
C:\Users\amitc\Documents\subham_work\SFS-QSAR-tool-master\loo.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  dataf['aver']=dataf[self.y.columns[0]].mean()
C:\Users\amitc\Documents\subham_work\SFS-QSAR-tool-master\loo.py:35: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has

In [8]:
fd['Average']=fd[["Q2LOO","R2Pred"]].mean(axis=1)
fd=fd.sort_values('Average', ascending=False)

In [9]:
fd

,random_seed,score,fold,Q2LOO,R2Pred,rm2tr,rm2ts,Average
1,42,r2,5,0.775126,0.784713,0.690070,0.702300,0.779919
0,42,r2,0,0.757618,0.752780,0.667870,0.645750,0.755199
7,42,neg_mean_gamma_deviance,5,0.735957,0.762723,0.639701,0.660424,0.749340
4,42,neg_mean_poisson_deviance,0,0.756455,0.736562,0.666869,0.623025,0.746509
1,20,r2,5,0.793502,0.683795,0.713147,0.507478,0.738648
5,42,neg_mean_poisson_deviance,5,0.762990,0.683353,0.674500,0.583284,0.723171
3,42,neg_mean_absolute_error,5,0.740237,0.696704,0.644529,0.572917,0.718471
6,42,neg_mean_gamma_deviance,0,0.747921,0.672154,0.654827,0.560180,0.710038
2,42,neg_mean_absolute_error,0,0.737878,0.678060,0.641749,0.570029,0.707969
5,20,neg_mean_poisson_deviance,5,0.834657,0.577601,0.768831,0.376017,0.706129


In [18]:
fd.to_csv('xyz.csv', index=False) #change output file name